In [ ]:
# from neuprint import Client

# c = Client('https://neuprint-cns.janelia.org/', dataset='optic-lobe', token= TBD)


import sys
from pathlib import Path

import numpy as np
import plotly.graph_objects as go

from dotenv import load_dotenv, find_dotenv


load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent

sys.path.append(str(PROJECT_ROOT.joinpath('src')))
print(f"Project root directory: {PROJECT_ROOT}")

from utils import olc_client



In [ ]:
import pandas as pd,numpy as np

from neuprint import fetch_synapses,NeuronCriteria as NC, SynapseCriteria as SC,merge_neuron_properties,IsNull, NotNull
from neuprint.queries import fetch_synapse_connections,fetch_neurons, fetch_adjacencies,fetch_synapses
from sklearn import decomposition
from collections import Counter

from scipy.cluster.hierarchy import fcluster
from scipy.spatial.distance import pdist
import fastcluster 

import plotly.express as px
import kaleido

In [ ]:
c = olc_client.connect(verbose=True)

data_dir = PROJECT_ROOT / "results" / "clustering"
cache_dir = PROJECT_ROOT / "cache" / "clustering"

In [ ]:
from utils.clustering_functions import remove_brackets, make_in_and_output_df, get_row_linkage, cluster_dict_from_linkage, set_annotations, make_count_table, cluster_with_type_names, format_list, set_pca_for_projections, get_combined_synapses_with_stdev, remove_R_or_L

In [ ]:
# make a directionary of bodyIds->cell types
# neuprint only version (i.e. no additional external annotations

# named optic lobe neurons from neuprint
criteria = NC(type=NotNull, rois=['ME(R)', 'LO(R)', 'LOP(R)','AME(R)','LA(R)'], roi_req='any')
named_neurons, _ = fetch_neurons(criteria)

#  dictionary bodyId:type
bodyId_type = dict(zip(list(named_neurons.bodyId.astype(int)),list(named_neurons.type)))

# list of  named cell types with arbors in both OL (i.e.a left and right instance in the dataset)
# purpose is to treat L and R instances a different types for clustering purposes
named_neurons['LR']=named_neurons.instance
named_neurons['LR']=named_neurons['LR'].apply(lambda x: x[-2:])  # assumes instances ending with '_L'or '_R'
LR_count = named_neurons.groupby(by=['type','LR'],as_index=False).count().groupby(by='type').count()
bilateral_cell_types = LR_count [LR_count ['LR']==2].index.tolist()


# dictionary of bodyId:instance for instances of bilateral types
bilateral_cells_bodyId_instance = dict(zip(named_neurons[named_neurons['type'].isin(bilateral_cell_types)].bodyId.tolist(),named_neurons[named_neurons['type'].isin(bilateral_cell_types)].instance.tolist()))

#bodies with named instances without a type name (fragments have an instance but not a type name)

criteria = NC(instance=NotNull, rois=['ME(R)', 'LO(R)', 'LOP(R)','AME(R)','LA(R)'], roi_req='any')
named_instances, _ = fetch_neurons(criteria)
named_instances= (named_instances[((named_instances['type']!= named_instances['type'])|
                                   (named_instances['type']==''))])   #. type either None or empty string
## named_instances.instance= named_instances.instance.apply(remove_brackets)  ##not needed?

named_instances.instance= named_instances.instance.apply(remove_R_or_L)

#   dictionary bodyId:instance for EM bodies without a type 
bodyId_instance = dict(zip(list(named_instances.bodyId.astype(int)),list(named_instances.instance)))

# combine dictionaries:
# replace entries for bilateral cells with bilateral_cells_bodyId_instance
bodyId_type = { **bodyId_type,**bilateral_cells_bodyId_instance} 
## add instances
bodyId_type = { **bodyId_instance,**bodyId_type}  ## instances don't overwrite what is already in dictionary

# remove None types (should not be needed anymore)
bodyId_type = {k: v for k, v in bodyId_type.items() if v is not None}  

## known false merges (dictionary) (only needed for versions used during proofreading stage)
false_merges_dict ={}

## types names in 'exclude from clustering' are not used for clustering
## in this version all cells with 'unclear' in name are not used for clustering
exclude_from_clustering =[x for x in list(set(bodyId_type.values())) if 'unclear' in x]
exclude_from_clustering =exclude_from_clustering +['(Lamina_TBD']  # temporary fix, remove later

# list of all unique names in use
type_and_instance_names = set(list (bodyId_type.values()))  

# dictionay of names to be changed when generating connectivity table for clustering
# here used to identify 'fragments' to be combined with tge corresponding types

names_with_fragment = [x for x in type_and_instance_names if 'fragment' in x]
fragment_type_dict = dict(zip(names_with_fragment,[x.split('_fragment')[0] for x in names_with_fragment]))
fragment_type_dict = {bodyId: name  for bodyId, name in fragment_type_dict.items()} 



# summary
names_with_unclear = [x for x in type_and_instance_names if 'unclear' in x]
names_with_fragment = [x for x in type_and_instance_names if 'fragment' in x]
print ('total cells', len(bodyId_type))
print ('total instance only names', len(bodyId_instance))
print ('total instance types', len(list(set(list (bodyId_instance.values())))))
print('all names in use', len(type_and_instance_names))
print('fragment names', len(names_with_fragment))
print('"unclear" names', len(names_with_unclear))
print('bilateral_types', len(bilateral_cell_types))
print('cell types', (len(type_and_instance_names)-len(names_with_fragment)
                     -len(names_with_unclear)-len(bilateral_cell_types)))

fragment_type_dict

In [ ]:
## list of bodyIds to cluster (here: based on region and synapse numbers and whether the body has type or instance name)

criteria = NC(rois=['ME(R)', 'LO(R)', 'LOP(R)','AME(R)','LA(R)'], roi_req='any')
neurons_all, _ = fetch_neurons(criteria)
neurons_all = neurons_all[neurons_all.synweight > 100]  ## the >100 threshold is ok for most OL cells except a few near the edges

neuron_selection= list(set(neurons_all.bodyId.tolist()+list(bodyId_type.keys())))  
len(neuron_selection)

In [ ]:
## get up- and downstream synaptic partners of all bodies in neuron_selection

cache_target_fn = cache_dir / "ROL_targets_df_neuprint_only_102023.pickle"

if cache_target_fn.is_file():
    ## load dataframes with connection data (faster than getting these from neuprint 
    ## and soon no further changes will be expected for the right optic lobe for now)
    conn_df_targets = pd.read_pickle(cache_target_fn)
else:
    criteria = NC(bodyId=neuron_selection)
    neuron_df1, conn_df1 = fetch_adjacencies(criteria,None,include_nonprimary=False) # targets
    conn_df_targets = merge_neuron_properties(neuron_df1, conn_df1)
    del neuron_df1, conn_df1
    ## save dataframes with connection data (reload is faster than getting these from neuprint)
    cache_dir.mkdir(exist_ok=True)
    conn_df_targets.to_pickle(cache_target_fn)



In [ ]:
cache_input_fn  = cache_dir / "ROL_inputs_df_neuprint_only_102023.pickle"

if cache_input_fn.is_file():
    ## load dataframes with connection data (faster than getting these from neuprint 
    ## and soon no further changes will be expected for the right optic lobe for now)
    conn_df_inputs = pd.read_pickle(cache_input_fn)
else:
    criteria = NC(bodyId=neuron_selection)
    neuron_df2, conn_df2 = fetch_adjacencies(None,criteria,include_nonprimary=False) # inputs
    conn_df_inputs = merge_neuron_properties(neuron_df2, conn_df2)
    del neuron_df2, conn_df2
    ## save dataframes with connection data (reload is faster than getting these from neuprint)
    cache_dir.mkdir(exist_ok=True)
    conn_df_inputs.to_pickle(cache_input_fn)


In [ ]:
### clustering a subset of neurons
### to run this for the full optic lobe set cell_list=neuron_selection (see above) and number_of_clusters to e.g. 600

### example: clusters MeTu4 cells and makes a table with the counts for each type in each cluster


## generate a list of bodyIds to cluster; for reclustering use clusters_bodyIds[] as input
type_selection = (list(set([cell_type for cell_type in bodyId_type.values() if (cell_type.startswith('MeTu4')) ])))
#type_selection =['LT51','LC33']


cell_list = [bodyId for bodyId in bodyId_type.keys() if  bodyId_type[bodyId] in type_selection]

## make connection table 
types_to_exclude= ( exclude_from_clustering)   # option to exclude cell types
named_types_for_clustering =[] # option to only use connections to selected types
 
connection_table =  (make_in_and_output_df(conn_df_inputs,conn_df_targets,bodyId_type,
                           types_to_exclude=types_to_exclude,
                           fragment_type_dict=fragment_type_dict,
                           bodyIds_to_use = cell_list))

# clustering:  produces dicitionaries of the bodyIds in each cluster, the types for each bodyId in a cluster 
# and the cluster it is in for each bodyIs

number_of_clusters = len(type_selection)-1  #-1 added in this case because MeTu4_unclear is not a separate type

row_linkage = get_row_linkage (connection_table,metric='cosine',linkage_method = 'ward',optimal_ordering=False)
clusters_bodyIds = cluster_dict_from_linkage (row_linkage,connection_table,t=number_of_clusters,criterion = 'maxclust')
clusters_cell_types = cluster_with_type_names(clusters_bodyIds,bodyId_type)

    
bodyIds_clusters =  {bodyId:cluster for cluster in clusters_bodyIds.keys() for bodyId in clusters_bodyIds[cluster]}

# table with results
cells_per_cluster_by_type = make_count_table (clusters_cell_types)
cells_per_cluster_by_type

In [ ]:
## example: bodyIds in a specific cluster

format_list(clusters_bodyIds[1])

In [ ]:
## example: cell_types in a specific cluster

format_list(clusters_cell_types[1])

In [ ]:

## The code below illustrates splitting cell types (or groups of cell types) into two groups and displaying the result
## as a map of cell locations (as mean synapse locations)

In [ ]:
## provides coordinate systems for plotting neuron postions in the main OL regions


pca_medulla= set_pca_for_projections (cell_type_pre = ['Mi1','Tm1'],cell_type_post = 'Pm2a')

pca_lobula = set_pca_for_projections (cell_type_pre = ['Tm5a','Tm5b-1'],cell_type_post = ['LC6','LC10c','LC16'],
                                     neuropile_region = 'LO(R)')


pca_lobula_plate = set_pca_for_projections (cell_type_pre = ['T4b','T5b'],cell_type_post = 'H2',neuropile_region = 'LOP(R)')

In [ ]:
conn_df_inputs

In [ ]:
## example: 

pca=pca_medulla  ##select  as appropriate for a  cell type
rois = ['ME(R)']
#rois = ['LO(R)']  
#pca=pca_lobula
#rois = ['LOP(R)'] 
#pca=pca_lobula_plate

type_names=['Tm5b']  ## cell type(s) to be subdivided
type_name='Tm5b'  # name to use for the group in output files

type_selection = (list(set([cell_type for cell_type in bodyId_type.values() if (cell_type in type_names) ])))
cell_list = [bodyId for bodyId in bodyId_type.keys() if  bodyId_type[bodyId] in type_selection]

total_cells = len(cell_list)
types_to_exclude= ( exclude_from_clustering)


connection_table =  (make_in_and_output_df(conn_df_inputs,conn_df_targets,bodyId_type,
                           types_to_exclude=types_to_exclude,
                           fragment_type_dict=fragment_type_dict,
                           bodyIds_to_use = cell_list))

number_of_clusters = 2
row_linkage = get_row_linkage (connection_table,metric='cosine',linkage_method = 'ward',optimal_ordering=False)
clusters_bodyIds = cluster_dict_from_linkage (row_linkage,connection_table,t=number_of_clusters,criterion = 'maxclust')
clusters_cell_types = cluster_with_type_names(clusters_bodyIds,bodyId_type)

total_cells = len(cell_list)

cells_per_cluster_by_type = make_count_table (clusters_cell_types)
print(cells_per_cluster_by_type)

cell_list1=clusters_bodyIds[1]

bodyIds_to_exclude = []  #list of relevant false merges
neuron_criteria = NC(bodyId=cell_list1)
synapse_criteria = SC(rois=rois, primary_only=False)

combined_synapses =get_combined_synapses_with_stdev(neuron_criteria,synapse_criteria,
                                                       bodyIds_to_exclude=bodyIds_to_exclude,rois=['LO(R)'],pca=pca)

cell_list2=clusters_bodyIds[2]
neuron_criteria = NC(bodyId=cell_list2)
combined_synapses2 =get_combined_synapses_with_stdev(neuron_criteria,synapse_criteria,
                                                       bodyIds_to_exclude=bodyIds_to_exclude,rois=['LO(R)'],pca=pca)

combined_synapses['type']=type_name+"-1"
combined_synapses2['type']=type_name+"-2"
synapses_for_plotting = pd.concat([combined_synapses,combined_synapses2])
synapses_for_plotting_2=synapses_for_plotting.copy()
synapses_for_plotting_2['type']=[type_name]*len(synapses_for_plotting_2)
synapses_for_plotting_2['facet']=['all']*len(synapses_for_plotting_2)
synapses_for_plotting['facet']=[type_name+' split']*len(synapses_for_plotting)

synapses_for_plotting = pd.concat([synapses_for_plotting_2,synapses_for_plotting])
#synapses_for_plotting.to_csv(directory+"synapses_for_subtype_plots/"+type_name+".csv")
                             
fig = px.scatter(synapses_for_plotting, x="X", y="Y", marginal_y=None,marginal_x=None,color='type',
                 hover_data=['bodyId','weight'],color_discrete_sequence=["blue", "red", "blue"], facet_col="facet")
    
fig.add_annotation(
    text=cells_per_cluster_by_type.to_string().replace('\n','<br>'), 
    align='left',
    showarrow=False,
    xref='paper',
    yref='paper',
    x=1.135,
    y=0.6,
    bordercolor='black',
    borderwidth=1)
fig.show()


In [ ]:
## split a series of cell types or cell type groups and save the results
## for 

type_names_types =[
    (['Tm5b-1','Tm5b-2'],'Tm5b',['ME(R)'] ),
    (['LPi34','LPi43'],'LPI34_LPi43',['LOP(R)'] ),
    (['LLPC1','LLPC2'],'LLPC1_LLPC2',['LOP(R)'] ),
    (['Li14','Li22'],'Li14_Li22',['LO(R)'] ),
    (['Pm2a','Pm2b'],'Pm2' ,['ME(R)']),
    (['LC10a','LC10a'],'LC10a',['LO(R)']),
    (['LC9'],'LC9', ['LO(R)']),
    (['LC6'],'LC6', ['LO(R)']),
    (['LC4'],'LC4', ['LO(R)']),
    (['LC11'],'LC11', ['LO(R)']),
    (['LC12'],'LC12', ['LO(R)']),
    (['Tm5b-1','Tm5b-2'],'Tm5b',['ME(R)'] ),
    (['Tm5a','Tm5b-1'],'Tm5ab',['ME(R)'] ),
    (['TmY9a','TmY9b'],'TmY9',['ME(R)'] ),
    (['Dm3a-1','Dm3a-2'],'Dm3a',['ME(R)'] ),
    (['Mti18a','Mti18b'],'Mti18ab',['ME(R)'] ),
    (['Pm5','Pm6'],'Pm5_Pm6',['ME(R)'] ),
    (['Mi10'],'Mi10',['ME(R)'] ),
    (['LC17'],'LC17', ['LO(R)'])
]

for type_names,type_name, rois in type_names_types:
    
    if rois==['ME(R)']:
        pca=pca_medulla
    elif rois==['LO(R)']:
        pca=pca_lobula
    else:
        pca=pca_lobula_plate
        
    type_selection = (list(set([cell_type for cell_type in bodyId_type.values() if (cell_type in type_names)])))
    cell_list = [bodyId for bodyId in bodyId_type.keys() if  bodyId_type[bodyId] in type_selection]

    total_cells = len(cell_list)
    types_to_exclude= (exclude_from_clustering)


    connection_table = (
        make_in_and_output_df(
            conn_df_inputs,
            conn_df_targets,
            bodyId_type,
            types_to_exclude=types_to_exclude,
            fragment_type_dict=fragment_type_dict,
            bodyIds_to_use = cell_list))

    number_of_clusters = 2
    row_linkage = get_row_linkage(
        connection_table,
        metric='cosine',
        linkage_method='ward',
        optimal_ordering=False)
    
    clusters_bodyIds = cluster_dict_from_linkage(
        row_linkage,
        connection_table,
        t=number_of_clusters,
        criterion = 'maxclust')
    clusters_cell_types = cluster_with_type_names(
        clusters_bodyIds,
        bodyId_type)

    total_cells = len(cell_list)

    cells_per_cluster_by_type = make_count_table (clusters_cell_types)
    print(cells_per_cluster_by_type)

    cell_list1=clusters_bodyIds[1]

    bodyIds_to_exclude = []  #list of relevant false merges
    neuron_criteria = NC(bodyId=cell_list1)
    synapse_criteria = SC(rois=rois, primary_only=False)

    combined_synapses =get_combined_synapses_with_stdev(
        neuron_criteria,
        synapse_criteria,
        bodyIds_to_exclude=bodyIds_to_exclude,
        rois=['LO(R)'])

    cell_list2=clusters_bodyIds[2]
    neuron_criteria = NC(bodyId=cell_list2)
    combined_synapses2 = get_combined_synapses_with_stdev(
        neuron_criteria,
        synapse_criteria,
        bodyIds_to_exclude=bodyIds_to_exclude,
        rois=['LO(R)'])

    combined_synapses['type']=type_name+"-1"
    combined_synapses2['type']=type_name+"-2"
    synapses_for_plotting = pd.concat([combined_synapses,combined_synapses2])
    synapses_for_plotting_2=synapses_for_plotting.copy()
    synapses_for_plotting_2['type']=[type_name]*len(synapses_for_plotting_2)
    synapses_for_plotting_2['facet']=['all']*len(synapses_for_plotting_2)
    synapses_for_plotting['facet']=[type_name+' split']*len(synapses_for_plotting)

    synapses_for_plotting = pd.concat([synapses_for_plotting_2,synapses_for_plotting])
    #synapses_for_plotting.to_csv(directory+"synapses_for_subtype_plots/"+type_name+".csv") 
                             
    fig = px.scatter(
        synapses_for_plotting, 
        x="X", y="Y", 
        marginal_y=None,
        marginal_x=None,
        color='type',
        hover_data=['bodyId','weight'],
        color_discrete_sequence=["blue", "red", "blue"], 
        facet_col="facet")
    
    fig.add_annotation(
        text=cells_per_cluster_by_type.to_string().replace('\n','<br>'), 
        align='left',
        showarrow=False,
        xref='paper',
        yref='paper',
        x=1.135,
        y=0.6,
        bordercolor='black',
        borderwidth=1)
    fig.show()
    
    ## saving figures as .html version (interactive) and pdfs
    output_dir = data_dir / "subtypes_plots"
    output_dir.mkdir(exist_ok=True)
    fig.write_html(output_dir / f"{type_name}_101923.html")
    fig.write_image(output_dir / f"{type_name}_101923.pdf", width=1080, height=540)


